<center><p>Multiclass varying k LSTM:</p></center>

In [146]:
import numpy as np

import keras.backend as K

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers import LSTM

<p>Our Training data, labels for the training data, and the sample query:</p>

In [190]:
#Create the training labels
train_y = (np.random.randint(0,150,10000))
y = np.zeros((10000,150))
c = 0
for i in train_y: 
    y[c][i] = 1
    c = c+1
train_y = y.reshape(10000,1,150).reshape(10000,150)

#Multi encoded training labels
r = np.random.rand(10000,150)
r[r <= 0.3] = False
r[r > 0.3] = True
train_y[r == False] = 1
train_y = train_y.reshape(10000,1,150)

In [213]:
#Create the training x data
train_x = np.array([np.random.rand(1, 1000)[0] for i in range(10000)]).reshape(10000,1,1000)

In [272]:
#Create the sample query 
sample = train_x[0].reshape(1,1,1000)

<p>Model Setup:</p>

In [215]:
def multiclass_loss(y_true, y_pred):
    EPS = 1e-5
    y_pred = K.clip(y_pred, EPS, 1 - EPS)
    return -K.mean((1 - y_true) * K.log(1 - y_pred) + y_true * K.log(y_pred))

In [194]:
shape = train_x.shape[2]
num_classes = 150

In [216]:
def create_model(shape,num_classes):
    model = Sequential()
    model.add(LSTM(32, input_shape=(None,shape), return_sequences=True))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss=multiclass_loss, optimizer='adam', metrics=['accuracy'])
    return model

<p>Fit the model with training and testing data:</p>

In [251]:
model = create_model(shape,num_classes)
history = model.fit(train_x, train_y,
              batch_size=100, epochs=50,
              verbose = 1)

Epoch 1/50
10000/10000 [==============================] - 3s - loss: 0.6273 - acc: 5.0000e-04       

<p>Obtain prediction:</p>

In [273]:
pred = model.predict(sample)[0][0]
#print(pred)
print('Our top prediction is: ',np.argmax(pred),"\n")
print("Indices of Top 3 predictions:")
top3 = pred.argsort()[-3:][::-1]
print(top3,"\n")
print("Indices where we have a 1:")
print(np.argwhere(train_y[0] == 1))
#print(pred)
#print(min(pred))
#print(max(pred))
#print(np.mean(pred))

Our top prediction is:  141 

Indices of Top 3 predictions:
[141  40 120] 

Indices where we have a 1:
[[  0   1]
 [  0   2]
 [  0   7]
 [  0   9]
 [  0  13]
 [  0  15]
 [  0  16]
 [  0  18]
 [  0  23]
 [  0  27]
 [  0  28]
 [  0  35]
 [  0  36]
 [  0  42]
 [  0  44]
 [  0  45]
 [  0  47]
 [  0  49]
 [  0  50]
 [  0  54]
 [  0  55]
 [  0  56]
 [  0  61]
 [  0  62]
 [  0  68]
 [  0  69]
 [  0  73]
 [  0  78]
 [  0  80]
 [  0  81]
 [  0  87]
 [  0  88]
 [  0  89]
 [  0  90]
 [  0  91]
 [  0  94]
 [  0  99]
 [  0 107]
 [  0 110]
 [  0 112]
 [  0 116]
 [  0 119]
 [  0 121]
 [  0 125]
 [  0 127]
 [  0 134]
 [  0 136]
 [  0 142]
 [  0 147]]
